In [9]:
import oandapyV20
import oandapyV20.endpoints.instruments as instruments
from datetime import datetime, timedelta
import pandas as pd

account_id = "101-004-6156530-001"
api_token = "54e5ffd4a1cf06318d9ba4f285170443-e2b154fcf3773f983c8e35be32492bcb"
api_url="api-fxpractice.oanda.com"

client = oandapyV20.API(access_token=api_token)

# Define the instrument and granularity
instrument = 'EUR_USD'  # EUR/USD currency pair
granularity = 'M15'  # 15-minute data

all_data = pd.DataFrame()
yest_date = (datetime.now() - timedelta(days=1)).strftime('%Y-%m-%d')

from_date = '2024-06-01'
last_date = yest_date

while last_date <= yest_date:

  params = {
    "granularity": granularity,
    "from":from_date,
    'count':5000
  }

  r = instruments.InstrumentsCandles(instrument=instrument, params=params)

  client.request(r)

  hist_data = pd.DataFrame()

  for candle in r.response['candles']:
      hist_data.at[candle['time'], 'open'] = candle['mid']['o']
      hist_data.at[candle['time'], 'high'] = candle['mid']['h']
      hist_data.at[candle['time'], 'low'] = candle['mid']['l']
      hist_data.at[candle['time'], 'close'] = candle['mid']['c']

  hist_data = hist_data.reset_index()

  # rename index to datetime
  hist_data = hist_data.rename(columns={'index': 'dt'})
  hist_data['dt'] = pd.to_datetime(hist_data['dt'])

  # append to all_data
  all_data = pd.concat([all_data, hist_data], ignore_index=True)

  # get the last date
  last_date = all_data['dt'].max()
  print(last_date)
  from_date = last_date

  try:
    last_date = last_date.strftime('%Y-%m-%d')
    from_date = from_date.strftime('%Y-%m-%d')
  except:
    pass



2024-08-13 22:45:00+00:00
2024-10-24 05:45:00+00:00
2025-01-08 06:45:00+00:00
2025-03-21 05:45:00+00:00
2025-06-03 05:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 20:45:00+00:00
2025-06-13 2

KeyboardInterrupt: 

In [10]:
all_data.to_csv('EURUSD_15min_data.csv', index=False)

In [ ]:
hist_data = all_data.drop_duplicates(subset='dt')

In [ ]:
# plot usnig plotly
import plotly.graph_objects as go

fig = go.Figure(data=[go.Candlestick(x=hist_data['dt'],
                open=hist_data['open'],
                high=hist_data['high'],
                low=hist_data['low'],
                close=hist_data['close'])])

fig.show()

In [ ]:
hist_data.to_csv('BTCUSD.csv', index=False)